<a href="https://colab.research.google.com/github/marciofag/chatbot_devs_ai/blob/main/chatbot_devs_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -U -q google-generativeai

In [9]:
#Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [10]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [26]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Título": "Introdução ao Python",
    "Conteúdo": "Python é uma linguagem de programação de alto nível, interpretada, de script, imperativa, orientada a objetos, funcional, de tipagem dinâmica e forte. É amplamente utilizada para desenvolvimento web, análise de dados, inteligência artificial e aprendizado de máquina."}

DOCUMENT2 = {
    "Título": "JavaScript Fundamentos",
    "Conteúdo": "JavaScript é uma linguagem de programação de alto nível, interpretada, de script, dinâmica, de tipagem dinâmica, multiparadigma (estruturada, orientada a objetos, imperativa e funcional). É amplamente utilizada para desenvolvimento web, especialmente no lado do cliente, através de navegadores web."}

DOCUMENT3 = {
    "Título": "Princípios de Design de Software",
    "Conteúdo": "Os princípios de design de software são diretrizes que ajudam os desenvolvedores a criar sistemas de software eficientes, escaláveis e fáceis de manter. Estes incluem princípios como a separação de preocupações, a inversão de dependências, a substituição de Liskov, a substituição de Bertrand Meyer, entre outros."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [27]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Introdução ao Python,Python é uma linguagem de programação de alto ...
1,JavaScript Fundamentos,JavaScript é uma linguagem de programação de a...
2,Princípios de Design de Software,Os princípios de design de software são diretr...


In [28]:
model = "models/embedding-001"

In [29]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [30]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Introdução ao Python,Python é uma linguagem de programação de alto ...,"[0.017181603, -0.050332334, -0.0101505015, 0.0..."
1,JavaScript Fundamentos,JavaScript é uma linguagem de programação de a...,"[0.007070073, -0.031909622, -0.014384449, 0.02..."
2,Princípios de Design de Software,Os princípios de design de software são diretr...,"[0.026053775, -0.018854206, -0.0048686666, 0.0..."


In [31]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [32]:
# Exemplo de uso
consulta = "Quais são os principais princípios de design de software?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Os princípios de design de software são diretrizes que ajudam os desenvolvedores a criar sistemas de software eficientes, escaláveis e fáceis de manter. Estes incluem princípios como a separação de preocupações, a inversão de dependências, a substituição de Liskov, a substituição de Bertrand Meyer, entre outros.


In [33]:
generation_config = {
  "temperature": 0.5,
  "candidate_count": 1
}

In [34]:
prompt = f"Reescreva esse texto de uma forma mais informal, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Os princípios de design de software são como dicas para desenvolvedores criarem sistemas de software que funcionam bem, podem crescer e são fáceis de consertar. Alguns desses princípios são:

* Dividir as coisas em partes menores, cada uma com sua própria função
* Fazer com que as partes dependam umas das outras da maneira certa
* Garantir que objetos semelhantes se comportem de maneira semelhante
* Permitir que objetos sejam substituídos por outros objetos semelhantes
